<a href="https://colab.research.google.com/github/BrianArnesto/masterThesis/blob/main/Master_Thesis_Fake_News_Indonesian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference:

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html
https://scikit-learn.org/stable/modules/sgd.html
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
https://github.com/leelaylay/TweetSemEval
https://towardsdatascience.com/yet-another-twitter-sentiment-analysis-part-1-tackling-class-imbalance-4d7a7f717d44
https://github.com/cbaziotis/datastories-semeval2017-task4


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from google.colab import drive
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
drive.mount('/content/drive')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
pip install emoji


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
'''
import glob
all_files = glob.glob("twitter*.tsv")
frame = pd.DataFrame()
list_ = []
for file_ in all_files:
    df = pd.read_csv(file_,index_col=None, sep='\t', header=None, names=['id','sentiment','text','to_delete'])
    list_.append(df.iloc[:,:-1])
df = pd.concat(list_)
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df.tail()
'''

'\nimport glob\nall_files = glob.glob("twitter*.tsv")\nframe = pd.DataFrame()\nlist_ = []\nfor file_ in all_files:\n    df = pd.read_csv(file_,index_col=None, sep=\'\t\', header=None, names=[\'id\',\'sentiment\',\'text\',\'to_delete\'])\n    list_.append(df.iloc[:,:-1])\ndf = pd.concat(list_)\ndf = df.drop_duplicates()\ndf = df.reset_index(drop=True)\ndf.tail()\n'

In [21]:
df = pd.read_csv('/content/drive/MyDrive/thesis/news3.csv')
df.head()


,Unnamed: 0,id,title,content,source_issue,tanggal,classification,label
0,0,10928,Cek Fakta: Hoaks Kabar Nikita Mirzani Meninggal Dunia Akibat Serangan Jantung pada 2 November 2022,"\nLiputan6.com, Jakarta - Kabar tentang artis Nikita Mirzani meninggal dunia akibat serangan jantung pada 2 November 2022 beredar di media sosial. Kabar tersebut disebarkan salah satu akun Facebook pada 2 November 2022.\nAkun Facebook tersebut mengunggah video dari situs berbagi video YouTube. Video tersebut berjudul ""Innalillahi, Nikita Mirzani 'Meninggal Dunia' Akibat Serangan Jantung"".\nGambar dalam video tersebut Nikita Mirzani tengah terbaring di ranjang rumah sakit, tangannya tampak diinfus. Di samping gambar tersebut, terdapat sejumlah orang tengah berkumpul di sekitar pemakaman.\nVideo tersebut kemudian diberi judul ""BREAKING NEWS ""NIKITA MIRZANI 'MENINGGAL DUNIA' AKIBAT SERANGAN JANTUNG"".\n""Bener apa bener sih NI berita?.."" tulis salah satu akun Facebook.\nKonten yang disebarkan akun Facebook tersebut telah beberapa kali direspons dan dikomentari warganet.\nBenarkah kabar Nikita Mirzani meninggal dunia akibat serangan jantung pada 2 November 2022? Berikut penelusurannya.\n",liputan6.com,2022-11-11,False Content,3
1,1,10927,Cek Fakta: Tidak Benar Tepung Beras Bisa Meredakan Demam pada Anak,"\nLiputan6.com, Jakarta - Klaim tentang menempelkan tepung beras di tubuh bisa meredakan demam pada anak beredar di media sosial. Klaim tersebut disebarkan salah satu akun Facebook pada 31 Oktober 2022 lalu.\nAkun Facebook tersebut mengunggah video berisi tata cara menggunakan tepung beras untuk menurunkan demam pada anak. Yang pertama, tepung beras dicampur dengan air hingga membentuk adonan. Adonan tepung beras tersebut kemudian ditempelkan dan diusapkan pada bagian dada dan perut anak.\n""Sekarang ini lagi musim demam,saya share pengalaman ya sama teman teman,andalan mama dulu kami kecil,sampai sekarang saya bikin ke anak anak,setidaknya pertolongan pertama buat anak anak demam, supaya gak kejang kejang dan step.\nK*m*k* V*v**nn* pernah panas 40 tapi masih aktif\nHabis bikin tepung jangan buka ac ya,ataupun kipas,waktu bikin juga jangan buka ac dan kipas.Please nonton video sampai habis🙏\nCukup tepung campur air putih aduk kira bisa di pegang.\nBikin sampai tepung kering ya,kalau kering tandanya hawa panas badan di tarik tepung.\nPointnya yg mesti di lamakan lihat di video.\nHabis bikin tepung kasi minyak kayu putih satu badan,lalu urut urut,kasi makan yg hangat hangat.\nObat tetap di minum,kalau demam bukan karena virus atau bakteri ,makaakan cepat sembuh\nSelamat mencoba😊🙏🙏🙏🙏\nSharing is caring😊,"" tulis salah satu akun Facebook.\nKonten yang disebarkan akun Facebook tersebut telah 11 ribu kali ditonton dan mendapat 161 komentar dari warganet.\nBenarkah menempelkan adonan tepung beras di tubuh bisa meredakan demam pada anak? Berikut penelusurannya.\n",liputan6.com,2022-11-10,False Content,3
2,2,10926,[SALAH] Menkes Budi Gunadi Sadikin: “Diabetes Mudah Diobati”,"Akun Facebook bernama TH K menyebarluaskan tautan serta gambar yang memperlihatkan menkes Budi Gunadi Sadikin sedang memberikan pidato. Selain itu, di gambar tersebut dilengkapi penjelasan bahwa pidato tersebut membahas tentang semua orang Indonesia yang kaget dengan fakta bahwa diabetes mudah diobati.\n\nTautan dan gambar yang diunggah pada 19 Juli 2022 tersebut telah dibagikan ulang sebanyak 153 kali, serta telah direspon oleh hampir 4,000 orang.",facebook.com,2022-11-10,Misleading Content,7
3,3,10925,Cek Fakta: Tidak Benar Video Wanita Tertua di Dunia Berusia 399 Tahun,"\nLiputan6.com, Jakarta - Sebuah video yang diklaim seorang wanita tertua di dunia berusia 399 tahun beredar di media sosial. Video tersebut disebarkan salah satu akun Facebook pada 18 April 2022.\nVideo berdurasi 59 detik itu memperlihatkan seseorang tengah berbaring di tempat tidur dengan kondisi yang sudah renta.\nSosok dalam video tersebut kemudian disebut-sebut sebagai wanita tertua di dunia berusia 399 tahun.\n"

In [22]:
df = df[['id','classification','content']]
df.head()

,id,classification,content
0,10928,False Content,"\nLiputan6.com, Jakarta - Kabar tentang artis Nikita Mirzani meninggal dunia akibat serangan jantung pada 2 November 2022 beredar di media sosial. Kabar tersebut disebarkan salah satu akun Facebook pada 2 November 2022.\nAkun Facebook tersebut mengunggah video dari situs berbagi video YouTube. Video tersebut berjudul ""Innalillahi, Nikita Mirzani 'Meninggal Dunia' Akibat Serangan Jantung"".\nGambar dalam video tersebut Nikita Mirzani tengah terbaring di ranjang rumah sakit, tangannya tampak diinfus. Di samping gambar tersebut, terdapat sejumlah orang tengah berkumpul di sekitar pemakaman.\nVideo tersebut kemudian diberi judul ""BREAKING NEWS ""NIKITA MIRZANI 'MENINGGAL DUNIA' AKIBAT SERANGAN JANTUNG"".\n""Bener apa bener sih NI berita?.."" tulis salah satu akun Facebook.\nKonten yang disebarkan akun Facebook tersebut telah beberapa kali direspons dan dikomentari warganet.\nBenarkah kabar Nikita Mirzani meninggal dunia akibat serangan jantung pada 2 November 2022? Berikut penelusurannya.\n"
1,10927,False Content,"\nLiputan6.com, Jakarta - Klaim tentang menempelkan tepung beras di tubuh bisa meredakan demam pada anak beredar di media sosial. Klaim tersebut disebarkan salah satu akun Facebook pada 31 Oktober 2022 lalu.\nAkun Facebook tersebut mengunggah video berisi tata cara menggunakan tepung beras untuk menurunkan demam pada anak. Yang pertama, tepung beras dicampur dengan air hingga membentuk adonan. Adonan tepung beras tersebut kemudian ditempelkan dan diusapkan pada bagian dada dan perut anak.\n""Sekarang ini lagi musim demam,saya share pengalaman ya sama teman teman,andalan mama dulu kami kecil,sampai sekarang saya bikin ke anak anak,setidaknya pertolongan pertama buat anak anak demam, supaya gak kejang kejang dan step.\nK*m*k* V*v**nn* pernah panas 40 tapi masih aktif\nHabis bikin tepung jangan buka ac ya,ataupun kipas,waktu bikin juga jangan buka ac dan kipas.Please nonton video sampai habis🙏\nCukup tepung campur air putih aduk kira bisa di pegang.\nBikin sampai tepung kering ya,kalau kering tandanya hawa panas badan di tarik tepung.\nPointnya yg mesti di lamakan lihat di video.\nHabis bikin tepung kasi minyak kayu putih satu badan,lalu urut urut,kasi makan yg hangat hangat.\nObat tetap di minum,kalau demam bukan karena virus atau bakteri ,makaakan cepat sembuh\nSelamat mencoba😊🙏🙏🙏🙏\nSharing is caring😊,"" tulis salah satu akun Facebook.\nKonten yang disebarkan akun Facebook tersebut telah 11 ribu kali ditonton dan mendapat 161 komentar dari warganet.\nBenarkah menempelkan adonan tepung beras di tubuh bisa meredakan demam pada anak? Berikut penelusurannya.\n"
2,10926,Misleading Content,"Akun Facebook bernama TH K menyebarluaskan tautan serta gambar yang memperlihatkan menkes Budi Gunadi Sadikin sedang memberikan pidato. Selain itu, di gambar tersebut dilengkapi penjelasan bahwa pidato tersebut membahas tentang semua orang Indonesia yang kaget dengan fakta bahwa diabetes mudah diobati.\n\nTautan dan gambar yang diunggah pada 19 Juli 2022 tersebut telah dibagikan ulang sebanyak 153 kali, serta telah direspon oleh hampir 4,000 orang."
3,10925,False Content,"\nLiputan6.com, Jakarta - Sebuah video yang diklaim seorang wanita tertua di dunia berusia 399 tahun beredar di media sosial. Video tersebut disebarkan salah satu akun Facebook pada 18 April 2022.\nVideo berdurasi 59 detik itu memperlihatkan seseorang tengah berbaring di tempat tidur dengan kondisi yang sudah renta.\nSosok dalam video tersebut kemudian disebut-sebut sebagai wanita tertua di dunia berusia 399 tahun.\n""The oldest woman in the world 399 years old 😱👏,"" tulis salah satu akun Facebook.\nKonten yang disebarkan akun Facebook tersebut telah 384 ribu kali ditonton dan mendapat 431 komentar dari warganet.\nBenarkah dalam video itu merupakan wanita tertua di dunia berusia 399 tahun? Berikut penelusurannya.\n"
4,10924,False Content,"\nLiputan6.com, Jakarta- Cek Fakta Liputan6.com mendapati klaim video kondisi Amerika

In [23]:
df = df.rename({'id': 'id', 'classification': 'sentiment','content': 'text'}, axis='columns')
df

,id,sentiment,text
0,10928,False Content,"\nLiputan6.com, Jakarta - Kabar tentang artis Nikita Mirzani meninggal dunia akibat serangan jantung pada 2 November 2022 beredar di media sosial. Kabar tersebut disebarkan salah satu akun Facebook pada 2 November 2022.\nAkun Facebook tersebut mengunggah video dari situs berbagi video YouTube. Video tersebut berjudul ""Innalillahi, Nikita Mirzani 'Meninggal Dunia' Akibat Serangan Jantung"".\nGambar dalam video tersebut Nikita Mirzani tengah terbaring di ranjang rumah sakit, tangannya tampak diinfus. Di samping gambar tersebut, terdapat sejumlah orang tengah berkumpul di sekitar pemakaman.\nVideo tersebut kemudian diberi judul ""BREAKING NEWS ""NIKITA MIRZANI 'MENINGGAL DUNIA' AKIBAT SERANGAN JANTUNG"".\n""Bener apa bener sih NI berita?.."" tulis salah satu akun Facebook.\nKonten yang disebarkan akun Facebook tersebut telah beberapa kali direspons dan dikomentari warganet.\nBenarkah kabar Nikita Mirzani meninggal dunia akibat serangan jantung pada 2 November 2022? Berikut penelusurannya.\n"
1,10927,False Content,"\nLiputan6.com, Jakarta - Klaim tentang menempelkan tepung beras di tubuh bisa meredakan demam pada anak beredar di media sosial. Klaim tersebut disebarkan salah satu akun Facebook pada 31 Oktober 2022 lalu.\nAkun Facebook tersebut mengunggah video berisi tata cara menggunakan tepung beras untuk menurunkan demam pada anak. Yang pertama, tepung beras dicampur dengan air hingga membentuk adonan. Adonan tepung beras tersebut kemudian ditempelkan dan diusapkan pada bagian dada dan perut anak.\n""Sekarang ini lagi musim demam,saya share pengalaman ya sama teman teman,andalan mama dulu kami kecil,sampai sekarang saya bikin ke anak anak,setidaknya pertolongan pertama buat anak anak demam, supaya gak kejang kejang dan step.\nK*m*k* V*v**nn* pernah panas 40 tapi masih aktif\nHabis bikin tepung jangan buka ac ya,ataupun kipas,waktu bikin juga jangan buka ac dan kipas.Please nonton video sampai habis🙏\nCukup tepung campur air putih aduk kira bisa di pegang.\nBikin sampai tepung kering ya,kalau kering tandanya hawa panas badan di tarik tepung.\nPointnya yg mesti di lamakan lihat di video.\nHabis bikin tepung kasi minyak kayu putih satu badan,lalu urut urut,kasi makan yg hangat hangat.\nObat tetap di minum,kalau demam bukan karena virus atau bakteri ,makaakan cepat sembuh\nSelamat mencoba😊🙏🙏🙏🙏\nSharing is caring😊,"" tulis salah satu akun Facebook.\nKonten yang disebarkan akun Facebook tersebut telah 11 ribu kali ditonton dan mendapat 161 komentar dari warganet.\nBenarkah menempelkan adonan tepung beras di tubuh bisa meredakan demam pada anak? Berikut penelusurannya.\n"
2,10926,Misleading Content,"Akun Facebook bernama TH K menyebarluaskan tautan serta gambar yang memperlihatkan menkes Budi Gunadi Sadikin sedang memberikan pidato. Selain itu, di gambar tersebut dilengkapi penjelasan bahwa pidato tersebut membahas tentang semua orang Indonesia yang kaget dengan fakta bahwa diabetes mudah diobati.\n\nTautan dan gambar yang diunggah pada 19 Juli 2022 tersebut telah dibagikan ulang sebanyak 153 kali, serta telah direspon oleh hampir 4,000 orang."
3,10925,False Content,"\nLiputan6.com, Jakarta - Sebuah video yang diklaim seorang wanita tertua di dunia berusia 399 tahun beredar di media sosial. Video tersebut disebarkan salah satu akun Facebook pada 18 April 2022.\nVideo berdurasi 59 detik itu memperlihatkan seseorang tengah berbaring di tempat tidur dengan kondisi yang sudah renta.\nSosok dalam video tersebut kemudian disebut-sebut sebagai wanita tertua di dunia berusia 399 tahun.\n""The oldest woman in the world 399 years old 😱👏,"" tulis salah satu akun Facebook.\nKonten yang disebarkan akun Facebook tersebut telah 384 ribu kali ditonton dan mendapat 431 komentar dari warganet.\nBenarkah dalam video itu merupakan wanita tertua di dunia berusia 399 tahun? Berikut penelusurannya.\n"
4,10924,False Content,"\nLiputan6.com, Jakarta- Cek Fakta Liputan6.com mendapati klaim video kondisi Amerika Serikat

Therea are 41705  tweets in the dataset

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10465 entries, 0 to 10464
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         10465 non-null  int64 
 1   sentiment  10465 non-null  object
 2   text       10465 non-null  object
dtypes: int64(1), object(2)
memory usage: 245.4+ KB


In [25]:
df['token_length'] = [len(x.split(" ")) for x in df.text]
max(df.token_length)

1163

In [26]:
df.loc[df.token_length.idxmax(),'text']


'“Freddy Budiman Bongkar Pejabat Terlibat Jaringan Narkoba, Ini Daftarnya”\nPOJOKSATU.id, JAKARTA – Detik-detik eksekusi mati terpidana kasus narkoba terhadap Freddy Budiman dkk ‘menggetarkan’ Tanah Air. Ada yang pro dan banyak yang kontra terhadap eksekusi mati ini. Mereka punya alasan masing-masing.\n\nTermasuk Harris Azhar, Koordinator Kontras. Dia menulis panjang cerita pengakuan Freddy Budiman, salah satu bandar kakap. Tulisan Harris atas pengakuan Freddy Budiman mengungkap fakta-fakta yang jarang diketahui banyak orang.\n\nTulisan Harris ini diposting ulang Ulil Abshar Abdalla pada Jumat (29/7/2016) dini hari. Berikut tulisan kesaksian Harris Azhar dari Freddy Budiman:\n\n“Cerita Busuk dari seorang Bandit”\n\nKesaksian bertemu Freddy Budiman di Lapas Nusa Kambangan (2014)\n\nDi tengah proses persiapan eksekusi hukuman mati yang ketiga dibawah pemerintahan Joko Widodo, saya menyakini bahwa pelaksanaan ini hanya untuk ugal-ugalan popularitas. Bukan karena upaya keadilan. Hukum yang

In [27]:
df.sentiment.value_counts()

Misleading Content     2624
False Context          2222
False Content          1779
Fabricated Content     1149
Manipulated Content    1139
Impostor Content        502
Clarification           381
False Connection        278
TRUE                    229
Satire                  162
Name: sentiment, dtype: int64

# Data Cleaning

## Spell Correlction 
it is a simple spell correction dealing with repeated characters such as “sooooo goooood”. If the same character is repeated more than two times, it shortens the repetition to two. For example “sooooo goooood” will be transformed as “soo good”. This is not a perfect solution since even after correction, in case of “soo”, it is not a correct spelling. But at least it will help to reduce feature space by making “sooo”, “soooo”, “sooooo” to the same word “soo”

In [28]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" } 

In [29]:
import codecs
import unidecode
import re
import spacy
nlp = spacy.load("en_core_web_sm")


def spacy_cleaner(text):
    try:
        decoded = unidecode.unidecode(codecs.decode(text, "unicode_escape"))
    except:
        decoded = unidecode.unidecode(text)
    apostrophe_handled = re.sub("’", "'", decoded)
    expanded = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in apostrophe_handled.split(" ")])
    parsed = nlp(expanded)
    final_tokens = []
    for t in parsed:
        if t.is_punct or t.is_space or t.like_num or t.like_url or str(t).startswith('@'):
            pass
        else:
            if t.lemma_ == '-PRON-':
                final_tokens.append(str(t))
            else:
                sc_removed = re.sub("[^a-zA-Z]", '', str(t.lemma_))
                if len(sc_removed) > 1:
                    final_tokens.append(sc_removed)
    joined = ' '.join(final_tokens)
    spell_corrected = re.sub(r'(.)\1+', r'\1\1', joined)
    return spell_corrected

In [30]:
pd.set_option('display.max_colwidth', None)
df.text[:10]

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           \nLiputan6.com, Jakarta - Kabar tentang artis Nikita Mirzani meninggal dunia akibat serangan jantung pada 2 November 2022 beredar di media sosial. Kabar tersebut disebarkan salah satu akun Facebook pada 2 November 2022.\nAkun Facebook tersebut mengunggah video dari situs berbagi video YouTube. Video tersebut berjudul "Innalillahi, Nikita Mirzani 'Meninggal Dunia' Akibat Serangan Jantung".\nGambar dalam video 

In [31]:
#Applied the custom cleaner
[spacy_cleaner(t) for t in df.text[:10]]


['Jakarta Kabar tentang artis Nikita Mirzani meninggal dunia akibat serangan jantung pada November beredar di medium sosial kabar tersebut disebarkan salah satu akun Facebook pada November Akun Facebook tersebut mengunggah video dari situs berbagi video YouTube video tersebut berjudul Innalillahi Nikita Mirzani Meninggal Dunia Akibat Serangan Jantung Gambar dalam video tersebut Nikita Mirzani tengah terbare di ranjang rumah sakit tangannya tampak diinfus Di samping gambar tersebut terdapat sejumlah orang tengah berkumpul di sekitar pemakaman Video tersebut kemudian diberi judul break NEWS NIKITA MIRZANI MENINGGAL DUNIA AKIBAT SERANGAN JANTUNG Bener apa bener sih NI berita tuli salah satu akun Facebook Konten yang disebarkan akun Facebook tersebut telah beberapa kali direspons dan dikomentari warganet Benarkah kabar Nikita Mirzani meninggal dunia akibat serangan jantung pada November Berikut penelusurannya',
 'Jakarta Klaim tentang menempelkan tepung beras di tubuh bisa meredakan demam 

In [32]:
#Apply the cleaner to all dataset
df['clean_text'] = [spacy_cleaner(t) for t in df.text]

<ipython-input-29-dedc28415dd6>:10: DeprecationWarning: invalid escape sequence '\_'
  decoded = unidecode.unidecode(codecs.decode(text, "unicode_escape"))


In [33]:
for i,t in enumerate(df.text):
    if '\m' in t:
        print(i,t)

In [34]:
df.text[1192]

'Akun Sadri Lanang (fb.com/100054456763723) pada 20 Januari 2022 mengunggah sebuah video yang memperlihatkan Ade Armando yang menyampaikan narasi:\n\n“olah berbasis laporan intelijen itu dalam bahasa saya. Jadi digambarkan bahwa sejak awal Covid-19 (akhir 2019) sampai Juni 2021. Jumlah tentara Cina yang masuk ke Indonesia dengan sepengetahuan dan restu Pemerintah Jokowi sudah mencapai 14 juta orang. Menurut laporan intelijen ini, gelombang kedatangan tentara Cina memanfaatkan kelengahan masyarakat Indonesia yang sedang terpusat perhatiannya pada pandemi”'

In [35]:
spacy_cleaner(df.text[1192])

'Akun Sadri Lanang pada Januari mengunggah sebuah video yang memperlihatkan Ade Armando yang menyampaikan narasi aolah berbasis laporan intelijen itu dalam bahasa saya Jadi digambarkan bahwa sejak awal Covid akhir sampai Juni Jumlah tentara Cina yang masuk ke Indonesia dengan sepengetahuan dan restu Pemerintah Jokowi sudah mencapai juta orang Menurut laporan intelijen ini gelombang kedatangan tentara Cina memanfaatkan kelengahan masyarakat Indonesia yang sedang terpusat perhatiannya pada pandemia'

# Imbalanced Learning

In [36]:
df.sentiment.value_counts()

Misleading Content     2624
False Context          2222
False Content          1779
Fabricated Content     1149
Manipulated Content    1139
Impostor Content        502
Clarification           381
False Connection        278
TRUE                    229
Satire                  162
Name: sentiment, dtype: int64

In [37]:
#Original Imbalanced Data
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier

tvec = TfidfVectorizer(stop_words=None, max_features=100000, ngram_range=(1, 3))
lr = LogisticRegression(max_iter=1000)
rf = RandomForestClassifier(max_depth=2, random_state=0)
neigh = KNeighborsClassifier(n_neighbors=3)
gnb = GaussianNB()
pct = Perceptron(tol=1e-3, random_state=0)
sgd = SGDClassifier(loss="hinge", penalty="l2", max_iter=1000)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score

def lr_cv(splits, X, Y, pipeline, average_method):
    
    kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=777)
    accuracy = []
    precision = []
    recall = []
    f1 = []
    for train, test in kfold.split(X, Y):
        lr_fit = pipeline.fit(X[train], Y[train])
        prediction = lr_fit.predict(X[test])
        scores = lr_fit.score(X[test],Y[test])
        
        accuracy.append(scores * 100)
        precision.append(precision_score(Y[test], prediction, average=average_method)*100)
        print('Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           ')
        print('precision:',precision_score(Y[test], prediction, average=None))
        recall.append(recall_score(Y[test], prediction, average=average_method)*100)
        print('recall:   ',recall_score(Y[test], prediction, average=None))
        f1.append(f1_score(Y[test], prediction, average=average_method)*100)
        print('f1 score: ',f1_score(Y[test], prediction, average=None))
        print('-'*50)

    print("accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
    print("precision: %.2f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
    print("recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
    print("f1 score: %.2f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))

In [38]:
from sklearn.pipeline import Pipeline
original_pipeline = Pipeline([
    ('vectorizer', tvec),
    ('classifier', lr)
])
lr_cv(5, df.clean_text, df.sentiment, original_pipeline, 'macro')

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [1.         0.54069767 0.         0.7752809  0.39232053 0.89583333
 0.48       0.41553637 0.         0.6       ]
recall:    [0.02631579 0.40434783 0.         0.7752809  0.52927928 0.43
 0.21052632 0.64190476 0.         0.06521739]
f1 score:  [0.05128205 0.46268657 0.         0.7752809  0.4506232  0.58108108
 0.29268293 0.50449102 0.         0.11764706]
--------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.75       0.57668712 0.         0.75069252 0.40677966 0.88888889
 0.55434783 0.43120393 0.         0.66666667]
recall:    [0.03947368 0.40869565 0.         0.76123596 0.54054054 0.56
 0.22368421 0.66857143 0.         0.08695652]
f1 score:  [0.075      0.4783715  0.         0.75592748 0.46421663 0.68711656
 0.31875    0.52427184 0.         0.15384615]
--------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [1.         0.55555556 0.         0.81231672 0.37373737 0.85416667
 0.55       0.4019025  0.         0.71428571]
recall:    [0.02631579 0.43668122 0.         0.77808989 0.5        0.40594059
 0.19298246 0.64380952 0.         0.10869565]
f1 score:  [0.05128205 0.48899756 0.         0.79483501 0.42774566 0.55033557
 0.28571429 0.49487555 0.         0.18867925]
--------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [1.         0.55263158 0.         0.80923077 0.39482759 0.87272727
 0.47413793 0.41970803 0.         0.75      ]
recall:    [0.01315789 0.45652174 0.         0.73876404 0.51460674 0.47524752
 0.24229075 0.65839695 0.         0.06521739]
f1 score:  [0.02597403 0.5        0.         0.77239354 0.44682927 0.61538462
 0.32069971 0.51263001 0.         0.12      ]
--------------------------------------------------
Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [1.         0.56497175 0.         0.79444444 0.38735818 0.88461538
 0.5        0.42912371 0.         0.        ]
recall:    [0.01298701 0.43478261 0.         0.8056338  0.53707865 0.46
 0.24122807 0.63428571 0.         0.        ]

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


With data as it is without any resampling, we can see that the precision is higher than the recall.

## Oversampling


In [39]:
#Random Over Sampler
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler

ROS_pipeline = make_pipeline(tvec, RandomOverSampler(random_state=777),lr )
SMOTE_pipeline_lr = make_pipeline(tvec, SMOTE(random_state=777),lr)
SMOTE_pipeline_rf = make_pipeline(tvec, SMOTE(random_state=777),rf)
SMOTE_pipeline_neigh = make_pipeline(tvec, SMOTE(random_state=777),neigh)
SMOTE_pipeline_gnb = make_pipeline(tvec, SMOTE(random_state=777),gnb)
SMOTE_pipeline_pct = make_pipeline(tvec, SMOTE(random_state=777),pct)
SMOTE_pipeline_sgd = make_pipeline(tvec, SMOTE(random_state=777),sgd)



In [40]:
from collections import Counter
from imblearn.over_sampling import SMOTE 

sm = SMOTE(random_state=42)
tvec = TfidfVectorizer(stop_words=None, max_features=100000, ngram_range=(1, 3))
df_tfidf = tvec.fit_transform(df.clean_text)
X_res, y_res = sm.fit_resample(df_tfidf, df.sentiment)
print('Distribution before SMOTE: {}'.format(Counter(df.sentiment)))
print('Distribution after SMOTE: {}'.format(Counter(y_res)))

Distribution before SMOTE: Counter({'Misleading Content': 2624, 'False Context': 2222, 'False Content': 1779, 'Fabricated Content': 1149, 'Manipulated Content': 1139, 'Impostor Content': 502, 'Clarification': 381, 'False Connection': 278, 'TRUE': 229, 'Satire': 162})
Distribution after SMOTE: Counter({'False Content': 2624, 'Misleading Content': 2624, 'Fabricated Content': 2624, 'Impostor Content': 2624, 'Manipulated Content': 2624, 'False Context': 2624, 'Satire': 2624, 'False Connection': 2624, 'TRUE': 2624, 'Clarification': 2624})


In [41]:
lr_cv(5, df.clean_text, df.sentiment, ROS_pipeline, 'macro')

Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.31578947 0.44672131 0.2        0.78962536 0.4407767  0.62352941
 0.35471698 0.51948052 0.33333333 0.36956522]
recall:    [0.31578947 0.47391304 0.12727273 0.76966292 0.51126126 0.53
 0.4122807  0.45714286 0.18181818 0.36956522]
f1 score:  [0.31578947 0.45991561 0.15555556 0.77951636 0.4734098  0.57297297
 0.38133874 0.48632219 0.23529412 0.36956522]
--------------------------------------------------
Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.2815534  0.48341232 0.15       0.79464286 0.44291339 0.68103448
 0.4        0.53469388 0.17857143 0.2745098 ]
recall:    [0.38157895 0.44347826 0.10909091 0.75       0.50675676 0.79
 0.36842105 0.49904762 0.15151515 0.30434783]
f1 sc

In [42]:
#SMOTE with Logistic Regression
lr_cv(5, df.clean_text, df.sentiment, SMOTE_pipeline_lr, 'macro')


Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.29268293 0.45217391 0.18918919 0.78947368 0.44327731 0.57291667
 0.33207547 0.51446281 0.20689655 0.34615385]
recall:    [0.31578947 0.45217391 0.12727273 0.75842697 0.47522523 0.55
 0.38596491 0.47428571 0.18181818 0.39130435]
f1 score:  [0.30379747 0.45217391 0.15217391 0.77363897 0.45869565 0.56122449
 0.35699797 0.49355798 0.19354839 0.36734694]
--------------------------------------------------
Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.26363636 0.49261084 0.20512821 0.80864198 0.46105263 0.51633987
 0.38461538 0.54545455 0.1875     0.23076923]
recall:    [0.38157895 0.43478261 0.14545455 0.73595506 0.49324324 0.79
 0.35087719 0.50285714 0.18181818 0.32608696]
f1 sc

In [43]:
#SMOTE with Random Forest
lr_cv(5, df.clean_text, df.sentiment, SMOTE_pipeline_rf, 'macro')


Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.16083916 0.35582822 0.04       0.67486339 0.30140187 0.39552239
 0.13333333 0.3559322  0.0456621  0.17045455]
recall:    [0.30263158 0.25217391 0.03636364 0.69382022 0.58108108 0.53
 0.00877193 0.04       0.3030303  0.32608696]
f1 score:  [0.21004566 0.29516539 0.03809524 0.68421053 0.39692308 0.45299145
 0.01646091 0.07191781 0.07936508 0.2238806 ]
--------------------------------------------------
Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.17177914 0.28244275 0.01538462 0.63231198 0.32820513 0.12793177
 0.33333333 0.5        0.03674541 0.09375   ]
recall:    [0.36842105 0.16086957 0.01818182 0.63764045 0.28828829 0.6
 0.00438596 0.00380952 0.42424242 0.26086957]
f1 sco

In [44]:
#SMOTE with KNN
lr_cv(5, df.clean_text, df.sentiment, SMOTE_pipeline_neigh, 'macro')


Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.08547009 0.53225806 0.05982906 0.41275797 0.35714286 0.29816514
 0.21383648 0.6        0.04398148 0.2       ]
recall:    [0.39473684 0.14347826 0.25454545 0.61797753 0.06756757 0.65
 0.14912281 0.01714286 0.57575758 0.02173913]
f1 score:  [0.14051522 0.2260274  0.09688581 0.49493813 0.11363636 0.40880503
 0.17571059 0.03333333 0.08172043 0.03921569]
--------------------------------------------------
Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.09792285 0.25443787 0.28571429 0.38039867 0.53623188 0.23398329
 0.14723926 1.         0.04289544 0.57142857]
recall:    [0.43421053 0.18695652 0.03636364 0.64325843 0.08333333 0.84
 0.10526316 0.01333333 0.48484848 0.08695652]
f1 sc

In [45]:
#SMOTE with Perceptron
lr_cv(5, df.clean_text, df.sentiment, SMOTE_pipeline_pct, 'macro')


Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.22       0.39351852 0.11111111 0.71204188 0.4009009  0.4587156
 0.37356322 0.44741533 0.13888889 0.15517241]
recall:    [0.14473684 0.36956522 0.12727273 0.76404494 0.4009009  0.5
 0.28508772 0.47809524 0.15151515 0.19565217]
f1 score:  [0.17460317 0.38116592 0.11864407 0.73712737 0.4009009  0.4784689
 0.32338308 0.46224678 0.14492754 0.17307692]
--------------------------------------------------
Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.17741935 0.41414141 0.07246377 0.74932615 0.44723618 0.63247863
 0.34615385 0.45819398 0.         0.21538462]
recall:    [0.14473684 0.35652174 0.09090909 0.78089888 0.4009009  0.74
 0.27631579 0.52190476 0.         0.30434783]
f1 score

In [46]:
#SMOTE with SGD
lr_cv(5, df.clean_text, df.sentiment, SMOTE_pipeline_sgd, 'macro')


Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.21052632 0.38110749 0.09615385 0.72916667 0.52159468 0.48623853
 0.36507937 0.55347594 0.13793103 0.28169014]
recall:    [0.36842105 0.50869565 0.18181818 0.78651685 0.3536036  0.53
 0.40350877 0.39428571 0.24242424 0.43478261]
f1 score:  [0.26794258 0.43575419 0.12578616 0.75675676 0.42147651 0.50717703
 0.38333333 0.46051168 0.17582418 0.34188034]
--------------------------------------------------
Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.20245399 0.41700405 0.12244898 0.76731302 0.46130952 0.55172414
 0.36744186 0.54707379 0.09259259 0.19753086]
recall:    [0.43421053 0.44782609 0.21818182 0.77808989 0.3490991  0.8
 0.34649123 0.40952381 0.15151515 0.34782609]
f1 sco

## Downsampling

In [47]:
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from imblearn import under_sampling as us

RUS_pipeline = make_pipeline(tvec, RandomUnderSampler(random_state=777),lr)
NM1_pipeline = make_pipeline(tvec, NearMiss(sampling_strategy='not minority', version = 1),lr)
NM2_pipeline = make_pipeline(tvec, NearMiss(sampling_strategy='not minority', version = 2),lr)
NM3_pipeline = make_pipeline(tvec, NearMiss(sampling_strategy='not minority', version = 3, n_neighbors_ver3=4),lr)

### RandomUnderSampler


In [48]:
lr_cv(5, df.clean_text, df.sentiment, RUS_pipeline, 'macro')


Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.14572864 0.39055794 0.08029197 0.76086957 0.47590361 0.44915254
 0.30057803 0.56666667 0.0787037  0.17105263]
recall:    [0.38157895 0.39565217 0.4        0.68820225 0.17792793 0.53
 0.22807018 0.25904762 0.51515152 0.56521739]
f1 score:  [0.21090909 0.39308855 0.1337386  0.72271386 0.25901639 0.48623853
 0.25935162 0.35555556 0.13654618 0.26262626]
--------------------------------------------------
Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.17209302 0.44390244 0.09659091 0.68468468 0.44166667 0.65178571
 0.304      0.53874539 0.05947955 0.14285714]
recall:    [0.48684211 0.39565217 0.30909091 0.64044944 0.23873874 0.73
 0.16666667 0.27809524 0.48484848 0.45652174]
f1 sc

## Near Miss

### Near Miss 1

In [49]:
lr_cv(5, df.clean_text, df.sentiment, NM1_pipeline, 'macro')


Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.09033613 0.39393939 0.06286837 0.80952381 0.48529412 0.4
 0.31007752 0.44318182 0.06849315 0.14465409]
recall:    [0.56578947 0.22608696 0.58181818 0.66853933 0.07432432 0.66
 0.1754386  0.07428571 0.15151515 0.5       ]
f1 score:  [0.1557971  0.28729282 0.11347518 0.73230769 0.12890625 0.49811321
 0.22408964 0.12724307 0.09433962 0.22439024]
--------------------------------------------------
Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.08348457 0.325      0.0616622  0.75       0.45689655 0.41052632
 0.38834951 0.48387097 0.075      0.12903226]
recall:    [0.60526316 0.16956522 0.41818182 0.65730337 0.11936937 0.78
 0.1754386  0.05714286 0.18181818 0.52173913]
f1 score:  [

## Near Miss 2

In [50]:
lr_cv(5, df.clean_text, df.sentiment, NM2_pipeline, 'macro')


Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.14864865 0.46540881 0.064      0.79530201 0.44347826 0.47826087
 0.23493976 0.51707317 0.06097561 0.14583333]
recall:    [0.43421053 0.32173913 0.43636364 0.66573034 0.11486486 0.55
 0.17105263 0.20190476 0.45454545 0.60869565]
f1 score:  [0.22147651 0.38046272 0.11162791 0.72477064 0.18246869 0.51162791
 0.19796954 0.29041096 0.10752688 0.23529412]
--------------------------------------------------
Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.1259542  0.43125    0.07536232 0.79411765 0.41578947 0.58461538
 0.26666667 0.54245283 0.05394191 0.14204545]
recall:    [0.43421053 0.3        0.47272727 0.60674157 0.17792793 0.76
 0.12280702 0.21904762 0.39393939 0.54347826]
f1 sc

### Near Miss 3

In [51]:
lr_cv(5, df.clean_text, df.sentiment, NM3_pipeline, 'macro')


/usr/local/lib/python3.8/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:175: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.09935897 0.35202492 0.06666667 0.         0.45945946 0.48
 0.26582278 0.37417219 0.06770833 0.13541667]
recall:    [0.40789474 0.49130435 0.45454545 0.         0.03828829 0.6
 0.27631579 0.2152381  0.39393939 0.56521739]
f1 score:  [0.15979381 0.41016334 0.11627907 0.         0.07068607 0.53333333
 0.27096774 0.2732769  0.11555556 0.21848739]
--------------------------------------------------


/usr/local/lib/python3.8/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:175: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.09004739 0.30372493 0.05445545 0.         0.37894737 0.
 0.22966507 0.41732283 0.08536585 0.12280702]
recall:    [0.5        0.46086957 0.4        0.         0.16216216 0.
 0.21052632 0.10095238 0.42424242 0.60869565]
f1 score:  [0.15261044 0.36614853 0.09586057 0.         0.22712934 0.
 0.21967963 0.16257669 0.14213198 0.20437956]
--------------------------------------------------


/usr/local/lib/python3.8/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:175: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.8/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:175: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.10601719 0.29501916 0.0472973  0.         0.36912752 0.48951049
 0.23529412 0.39754098 0.05128205 0.15422886]
recall:    [0.48684211 0.33478261 0.25       0.         0.12359551 0.69306931
 0.26431718 0.1851145  0.3125     0.67391304]
f1 score:  [0.17411765 0.31364562 0.07954545 0.         0.18518519 0.57377049
 0.24896266 0.25260417 0.08810573 0.25101215]
--------------------------------------------------


/usr/local/lib/python3.8/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:175: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(


Misleading Content,  False Context,  False Content, Fabricated Content, Manipulated Content, Impostor Content, Clarification, False Connection, TRUE, Satire           
precision: [0.09090909 0.35014006 0.05093834 0.         0.3304721  0.
 0.21800948 0.5049505  0.05797101 0.10362694]
recall:    [0.49350649 0.54347826 0.33928571 0.         0.17303371 0.
 0.20175439 0.09714286 0.375      0.44444444]
f1 score:  [0.15353535 0.42589438 0.08857809 0.         0.22713864 0.
 0.2095672  0.1629393  0.10041841 0.16806723]
--------------------------------------------------
accuracy: 19.53% (+/- 1.89%)
precision: 18.93% (+/- 2.75%)
recall: 29.91% (+/- 3.31%)
f1 score: 17.78% (+/- 3.21%)


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
from collections import Counter

nm3 = NearMiss(sampling_strategy='not minority', version=3, n_neighbors_ver3=4)
tvec = TfidfVectorizer(stop_words=None, max_features=100000, ngram_range=(1, 3))
df_tfidf = tvec.fit_transform(df.clean_text)
X_res, y_res = nm3.fit_resample(df_tfidf, df.sentiment)
print('Distribution before NearMiss-3: {}'.format(Counter(df.sentiment)))
print('Distribution after NearMiss-3: {}'.format(Counter(y_res)))

Distribution before NearMiss-3: Counter({'Misleading Content': 2624, 'False Context': 2222, 'False Content': 1779, 'Fabricated Content': 1149, 'Manipulated Content': 1139, 'Impostor Content': 502, 'Clarification': 381, 'False Connection': 278, 'TRUE': 229, 'Satire': 162})
Distribution after NearMiss-3: Counter({'Clarification': 162, 'Fabricated Content': 162, 'False Connection': 162, 'False Context': 162, 'Manipulated Content': 162, 'Satire': 162, 'TRUE': 162, 'Misleading Content': 139, 'False Content': 10, 'Impostor Content': 4})


/usr/local/lib/python3.8/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:175: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(


In [53]:
from collections import Counter

nm2 = NearMiss(sampling_strategy='not minority', version=2, n_neighbors_ver3=4)
tvec = TfidfVectorizer(stop_words=None, max_features=100000, ngram_range=(1, 3))
df_tfidf = tvec.fit_transform(df.clean_text)
X_res, y_res = nm2.fit_resample(df_tfidf, df.sentiment)
print('Distribution before NearMiss-2: {}'.format(Counter(df.sentiment)))
print('Distribution after NearMiss-2: {}'.format(Counter(y_res)))

Distribution before NearMiss-2: Counter({'Misleading Content': 2624, 'False Context': 2222, 'False Content': 1779, 'Fabricated Content': 1149, 'Manipulated Content': 1139, 'Impostor Content': 502, 'Clarification': 381, 'False Connection': 278, 'TRUE': 229, 'Satire': 162})
Distribution after NearMiss-2: Counter({'Clarification': 162, 'Fabricated Content': 162, 'False Connection': 162, 'False Content': 162, 'False Context': 162, 'Impostor Content': 162, 'Manipulated Content': 162, 'Misleading Content': 162, 'Satire': 162, 'TRUE': 162})


In [54]:
from collections import Counter

nm1 = NearMiss(sampling_strategy='not minority', version=1, n_neighbors_ver3=4)
tvec = TfidfVectorizer(stop_words=None, max_features=100000, ngram_range=(1, 3))
df_tfidf = tvec.fit_transform(df.clean_text)
X_res, y_res = nm1.fit_resample(df_tfidf, df.sentiment)
print('Distribution before NearMiss-1: {}'.format(Counter(df.sentiment)))
print('Distribution after NearMiss-1: {}'.format(Counter(y_res)))

Distribution before NearMiss-1: Counter({'Misleading Content': 2624, 'False Context': 2222, 'False Content': 1779, 'Fabricated Content': 1149, 'Manipulated Content': 1139, 'Impostor Content': 502, 'Clarification': 381, 'False Connection': 278, 'TRUE': 229, 'Satire': 162})
Distribution after NearMiss-1: Counter({'Clarification': 162, 'Fabricated Content': 162, 'False Connection': 162, 'False Content': 162, 'False Context': 162, 'Impostor Content': 162, 'Manipulated Content': 162, 'Misleading Content': 162, 'Satire': 162, 'TRUE': 162})
